Ah! That usually happens because of **how the YouTube API returns descriptions**. Some common reasons:

1. **Playlist snippet doesn’t include full descriptions**

   * `playlistItems.list` only returns **short snippets** of the description.
   * To get the **full description**, you need to call `videos.list` with the video ID and `part=snippet`.

2. **API quota or missing fields**

   * If `snippet["description"]` is missing, the expander will be empty.

---

Here’s how to **fix it** in your Streamlit app:

### ✅ Step 1: Use `videos.list` to fetch full description

```python
def get_video_description(api_key, video_id):
    url = "https://www.googleapis.com/youtube/v3/videos"
    params = {
        "part": "snippet",
        "id": video_id,
        "key": api_key
    }
    response = requests.get(url, params=params).json()
    items = response.get("items", [])
    if items:
        return items[0]["snippet"].get("description", "No description available.")
    return "No description found."
```

### ✅ Step 2: Call this function for the selected video

```python
description = get_video_description(API_KEY, selected_video["id"])
with st.expander("📝 Video Description", expanded=True):
    st.write(description)
```

---

### ⚡ Explanation

* `playlistItems.list` → only short info (sometimes empty description)
* `videos.list` → fetches **full snippet**, including complete `title`, `description`, `tags`, etc.
* Always use `videos.list` if you want **accurate descriptions** or extra metadata.

---

If you want, I can rewrite your **entire Streamlit playlist app** so that **title, full description, and top comments** all show correctly without missing data, fully ready to run.

Do you want me to do that?


Perfect! You want a **two-level Streamlit app**:

1. **Home page:** show all playlists as **cards** with thumbnail + title
2. **Playlist page:** when you click a playlist, show **all videos** (like before) with embedded player, description, comments

We can do this **entirely in Streamlit** using **session state** to track which playlist is selected.

---

## ✅ Step 1: Prepare Data

For each playlist, you need:

* Playlist ID
* Playlist title
* Playlist thumbnail (from playlist snippet)

You can fetch multiple playlists using the **YouTube Data API** (`playlists.list` for a channel).

---

## ✅ Step 2: Full Streamlit Example

```python
import requests
import streamlit as st

# --- Streamlit page config ---
st.set_page_config(page_title="YouTube Playlists Dashboard", layout="wide")

API_KEY = "YOUR_API_KEY"

# Example: your playlists (replace with real playlist IDs)
PLAYLISTS = [
    {"id": "PLbdsuU_MGf0cshTlm6fADLaGco4jF5V9N"},
    {"id": "PLabcd123"},
]

# --- Function to get playlist details ---
@st.cache_data
def get_playlist_details(api_key, playlist_id):
    url = "https://www.googleapis.com/youtube/v3/playlists"
    params = {"part": "snippet", "id": playlist_id, "key": api_key}
    response = requests.get(url, params=params).json()
    items = response.get("items", [])
    if not items:
        return None
    snippet = items[0]["snippet"]
    return {
        "title": snippet["title"],
        "thumbnail": snippet["thumbnails"]["medium"]["url"]
    }

# --- Function to get videos in a playlist ---
@st.cache_data
def get_playlist_videos(api_key, playlist_id):
    url = "https://www.googleapis.com/youtube/v3/playlistItems"
    params = {"part": "snippet", "playlistId": playlist_id, "maxResults": 50, "key": api_key}
    response = requests.get(url, params=params).json()
    videos = []
    for item in response.get("items", []):
        snippet = item["snippet"]
        videos.append({
            "id": snippet["resourceId"]["videoId"],
            "title": snippet["title"],
            "description": snippet.get("description", "")
        })
    return videos

# --- Session state to track current playlist ---
if "selected_playlist" not in st.session_state:
    st.session_state.selected_playlist = None

# --- Main Page ---
if st.session_state.selected_playlist is None:
    st.title("📚 My YouTube Playlists")

    cols = st.columns(3)
    for idx, pl in enumerate(PLAYLISTS):
        details = get_playlist_details(API_KEY, pl["id"])
        if details:
            col = cols[idx % 3]
            with col:
                st.image(details["thumbnail"], use_container_width=True)
                if st.button(details["title"], key=pl["id"]):
                    st.session_state.selected_playlist = pl["id"]
else:
    playlist_id = st.session_state.selected_playlist
    st.button("⬅ Back to Playlists", on_click=lambda: st.session_state.update({"selected_playlist": None}))

    # Show videos for selected playlist
    videos = get_playlist_videos(API_KEY, playlist_id)
    st.subheader("Videos in this Playlist")

    for video in videos:
        video_url = f"https://www.youtube.com/watch?v={video['id']}"
        st.markdown(f"### [{video['title']}]({video_url})")
        st.video(video_url)
        st.write(video["description"])
        st.divider()
```

---

## 🧠 How It Works

1. **Home page:**

   * Loops through your playlists
   * Shows **thumbnail + title** as a clickable card
   * Clicking a playlist sets `st.session_state.selected_playlist`

2. **Playlist page:**

   * Displays all videos for the selected playlist
   * Shows **title, embedded video, description**
   * Includes a **back button** to return to the playlist view

3. **Caching:**

   * `@st.cache_data` prevents repeated API calls

---

## ✅ Next Steps / Enhancements

* Add **top comments** for each video (like previous code)
* Show **video thumbnails** in playlist view
* Add **search or filter** in sidebar for videos
* Make the playlist cards **more visually appealing** using `st.markdown` with custom HTML

---

If you want, I can make a **fully polished “card UI” version** with playlist thumbnails, titles, and hover effects — just like a real course dashboard, all in Streamlit.

Do you want me to do that?


Absolutely! You can **host your Streamlit app live** while keeping your API key secure by **not hardcoding it** in your script. Instead, you use **environment variables** or a **secret manager**. Here’s a step-by-step guide:

---

## **1️⃣ Remove API Key from Code**

Instead of:

```python
API_KEY = "YOUR_API_KEY"
```

Use **Streamlit secrets** or environment variables.

---

### **Option A: Streamlit Secrets (Recommended for Streamlit Cloud)**

1. Create a file called `secrets.toml` in your project folder:

```toml
# secrets.toml
YOUTUBE_API_KEY = "YOUR_API_KEY"
```

2. Access it in your app:

```python
import streamlit as st

API_KEY = st.secrets["YOUTUBE_API_KEY"]
```

✅ The API key is now **never exposed in your code**.

---

### **Option B: Environment Variable**

Set an environment variable on your hosting platform:

```bash
export YOUTUBE_API_KEY="YOUR_API_KEY"
```

In Python:

```python
import os
API_KEY = os.getenv("YOUTUBE_API_KEY")
```

---

## **2️⃣ Host on Streamlit Cloud**

1. Push your app to a **GitHub repo** (do not commit your `secrets.toml`)
2. Go to [Streamlit Cloud](https://streamlit.io/cloud) and **deploy your app from GitHub**
3. In **Settings → Secrets**, add `YOUTUBE_API_KEY`

✅ Streamlit Cloud injects it securely into your app.

---

## **3️⃣ Optional: Host Elsewhere**

* **Heroku:** Use config vars (`heroku config:set YOUTUBE_API_KEY=...`)
* **AWS/GCP/Azure:** Use environment variables or secret managers
* **Docker:** Use `-e YOUTUBE_API_KEY=...` in `docker run`

---

## **4️⃣ Extra Tips for Security**

* Never push your API key to GitHub.
* Limit your API key in **Google Cloud Console**: restrict it to your app domain or IP addresses.
* Monitor quota usage in YouTube API dashboard to detect abuse.

---

If you want, I can **rewrite your Streamlit app fully ready for deployment on Streamlit Cloud**, with **secure API key** and all your features (multiple playlists, video player, descriptions, comments).

Do you want me to do that?


Perfect! You want to **turn your Streamlit app into a mini YouTube course management system** with:

1. **Admin panel:** Upload/manage playlists, store metadata in SQLite
2. **Course table:** Map course names to playlists or specific YouTube links
3. **Frontend:** Display playlists/videos in player dynamically

I can outline a **full working approach with code** using **Streamlit + SQLite + YouTube API**.

---

## **1️⃣ Setup SQLite database**

Create `database.py`:

```python
import sqlite3

def get_connection():
    conn = sqlite3.connect("youtube_courses.db")
    return conn

def init_db():
    conn = get_connection()
    cursor = conn.cursor()
    
    # Table for playlists
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS playlists (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            playlist_id TEXT UNIQUE,
            playlist_name TEXT,
            youtube_link TEXT,
            extract_link TEXT,
            batch TEXT
        )
    """)
    
    # Table for courses (manual mapping)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS courses (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            course_name TEXT,
            youtube_link TEXT
        )
    """)
    
    conn.commit()
    conn.close()

def insert_playlist(playlist_id, playlist_name, youtube_link, extract_link, batch):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("""
        INSERT OR IGNORE INTO playlists (playlist_id, playlist_name, youtube_link, extract_link, batch)
        VALUES (?, ?, ?, ?, ?)
    """, (playlist_id, playlist_name, youtube_link, extract_link, batch))
    conn.commit()
    conn.close()

def insert_course(course_name, youtube_link):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO courses (course_name, youtube_link)
        VALUES (?, ?)
    """, (course_name, youtube_link))
    conn.commit()
    conn.close()

def fetch_playlists():
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM playlists")
    data = cursor.fetchall()
    conn.close()
    return data

def fetch_courses():
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM courses")
    data = cursor.fetchall()
    conn.close()
    return data
```

> This sets up your SQLite DB with **playlists** and **courses** tables.

---

## **2️⃣ Admin panel in Streamlit**

```python
import streamlit as st
from database import init_db, insert_playlist, insert_course, fetch_playlists, fetch_courses

init_db()

st.sidebar.title("🔧 Admin Panel")
mode = st.sidebar.selectbox("Select Mode", ["Add Playlist", "Add Course", "View Data"])

if mode == "Add Playlist":
    st.header("➕ Add Playlist")
    playlist_id = st.text_input("Playlist ID")
    playlist_name = st.text_input("Playlist Name")
    youtube_link = st.text_input("YouTube Link")
    extract_link = st.text_input("Extract Link (optional)")
    batch = st.text_input("Batch")

    if st.button("Add Playlist"):
        if playlist_id and playlist_name:
            insert_playlist(playlist_id, playlist_name, youtube_link, extract_link, batch)
            st.success("Playlist added successfully!")
        else:
            st.error("Playlist ID and Name are required.")

elif mode == "Add Course":
    st.header("➕ Add Course")
    course_name = st.text_input("Course Name")
    youtube_link = st.text_input("YouTube Video or Playlist Link")
    if st.button("Add Course"):
        if course_name and youtube_link:
            insert_course(course_name, youtube_link)
            st.success("Course added successfully!")
        else:
            st.error("All fields are required.")

elif mode == "View Data":
    st.header("📋 Playlists")
    playlists = fetch_playlists()
    st.dataframe(playlists)

    st.header("📋 Courses")
    courses = fetch_courses()
    st.dataframe(courses)
```

---

## **3️⃣ Frontend player page**

```python
import streamlit as st
from database import fetch_courses
import requests

st.title("📘 Course Player")

courses = fetch_courses()
course_names = [c[1] for c in courses]
selected_course = st.selectbox("Select Course", course_names)

if selected_course:
    course = next(c for c in courses if c[1] == selected_course)
    youtube_link = course[2]

    # If playlist link, you can extract ID and show sidebar videos (reuse your previous code)
    st.video(youtube_link)
```

---

## **4️⃣ Key Features**

* **Admin Panel**

  * Add playlists with metadata (`playlist_id`, `playlist_name`, `youtube_link`, `batch`)
  * Add courses manually linking to YouTube videos or playlists
  * View current playlists/courses

* **Frontend**

  * Select course from dropdown
  * Plays video or playlist in embedded player
  * Can integrate your previous playlist sidebar code for full playlist support

* **SQLite**

  * Persistent local storage
  * Easy to extend (e.g., add more metadata like description, comments count)

---

I can now **combine this with your multi-playlist sidebar + video player** so that:

1. Admin adds playlists/courses
2. Frontend dynamically loads playlists and videos from SQLite
3. Each playlist shows **videos, description, comments**

This would become a **full mini-LMS for YouTube courses**.

Do you want me to build that full combined Streamlit app?


In [ ]:
Perfect! Let’s build a **full mini-LMS using Streamlit + YouTube API + SQLite**. This app will have:

1. **Admin Panel:** Add playlists, courses, manage data
2. **Frontend Player:** Select course → view playlist or video → show video + description + comments
3. **SQLite database:** Persistent storage for playlists and courses
4. **Secure API key:** Using `secrets.toml`

I’ll provide a **ready-to-run structure** with clear instructions.

---

## **Folder Structure**

```
yt_lms/
│
├─ app.py                # Main Streamlit app
├─ database.py           # SQLite DB helper
├─ secrets.toml          # API key (local)
└─ requirements.txt      # Dependencies
```

---

## **1️⃣ secrets.toml**

Create `.streamlit/secrets.toml` in your project folder:

```toml
YOUTUBE_API_KEY = "YOUR_YOUTUBE_API_KEY_HERE"
```

> Never push this to GitHub.

---

## **2️⃣ database.py**

```python
import sqlite3

def get_connection():
    conn = sqlite3.connect("youtube_courses.db")
    return conn

def init_db():
    conn = get_connection()
    cursor = conn.cursor()
    
    # Table for playlists
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS playlists (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            playlist_id TEXT UNIQUE,
            playlist_name TEXT,
            youtube_link TEXT,
            extract_link TEXT,
            batch TEXT
        )
    """)
    
    # Table for courses (manual mapping)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS courses (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            course_name TEXT,
            youtube_link TEXT
        )
    """)
    
    conn.commit()
    conn.close()

def insert_playlist(playlist_id, playlist_name, youtube_link, extract_link, batch):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("""
        INSERT OR IGNORE INTO playlists (playlist_id, playlist_name, youtube_link, extract_link, batch)
        VALUES (?, ?, ?, ?, ?)
    """, (playlist_id, playlist_name, youtube_link, extract_link, batch))
    conn.commit()
    conn.close()

def insert_course(course_name, youtube_link):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO courses (course_name, youtube_link)
        VALUES (?, ?)
    """, (course_name, youtube_link))
    conn.commit()
    conn.close()

def fetch_playlists():
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM playlists")
    data = cursor.fetchall()
    conn.close()
    return data

def fetch_courses():
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM courses")
    data = cursor.fetchall()
    conn.close()
    return data
```

---

## **3️⃣ app.py (Streamlit app)**

```python
import streamlit as st
import requests
from database import init_db, insert_playlist, insert_course, fetch_playlists, fetch_courses

# Initialize DB
init_db()

# Streamlit Page
st.set_page_config(page_title="YouTube LMS", layout="wide")
API_KEY = st.secrets["YOUTUBE_API_KEY"]

# --- Sidebar for Admin / Frontend ---
mode = st.sidebar.selectbox("Mode", ["Frontend", "Admin Panel"])

# ---------------- Admin Panel ----------------
if mode == "Admin Panel":
    st.title("🔧 Admin Panel")

    admin_tab = st.sidebar.radio("Admin Options", ["Add Playlist", "Add Course", "View Data"])

    if admin_tab == "Add Playlist":
        st.header("➕ Add Playlist")
        playlist_id = st.text_input("Playlist ID")
        playlist_name = st.text_input("Playlist Name")
        youtube_link = st.text_input("YouTube Link")
        extract_link = st.text_input("Extract Link (optional)")
        batch = st.text_input("Batch")

        if st.button("Add Playlist"):
            if playlist_id and playlist_name:
                insert_playlist(playlist_id, playlist_name, youtube_link, extract_link, batch)
                st.success("Playlist added successfully!")
            else:
                st.error("Playlist ID and Name are required.")

    elif admin_tab == "Add Course":
        st.header("➕ Add Course")
        course_name = st.text_input("Course Name")
        youtube_link = st.text_input("YouTube Video or Playlist Link")

        if st.button("Add Course"):
            if course_name and youtube_link:
                insert_course(course_name, youtube_link)
                st.success("Course added successfully!")
            else:
                st.error("All fields are required.")

    elif admin_tab == "View Data":
        st.header("📋 Playlists")
        playlists = fetch_playlists()
        st.dataframe(playlists)

        st.header("📋 Courses")
        courses = fetch_courses()
        st.dataframe(courses)

# ---------------- Frontend ----------------
else:
    st.title("📘 YouTube LMS Player")

    # Fetch courses
    courses = fetch_courses()
    if not courses:
        st.warning("No courses available. Ask admin to add courses.")
    else:
        course_names = [c[1] for c in courses]
        selected_course = st.selectbox("Select Course", course_names)

        if selected_course:
            course = next(c for c in courses if c[1] == selected_course)
            youtube_link = course[2]

            # Check if it's a playlist or single video
            if "playlist?list=" in youtube_link:
                playlist_id = youtube_link.split("list=")[1].split("&")[0]

                # Fetch playlist videos
                def get_playlist_videos(api_key, playlist_id):
                    url = f"https://www.googleapis.com/youtube/v3/playlistItems"
                    params = {
                        "part": "snippet",
                        "maxResults": 50,
                        "playlistId": playlist_id,
                        "key": api_key
                    }
                    response = requests.get(url, params=params).json()
                    videos = []
                    for item in response.get("items", []):
                        snippet = item["snippet"]
                        video_id = snippet["resourceId"]["videoId"]
                        title = snippet["title"]
                        videos.append({"id": video_id, "title": title})
                    return videos

                videos = get_playlist_videos(API_KEY, playlist_id)

                selected_video_title = st.selectbox("Select Video", [v["title"] for v in videos])
                selected_video = next(v for v in videos if v["title"] == selected_video_title)
                st.video(f"https://www.youtube.com/watch?v={selected_video['id']}")

            else:
                # Single video
                st.video(youtube_link)
```

---

## **4️⃣ Install dependencies**

```bash
pip install streamlit requests
```

---

## **5️⃣ Run locally**

```bash
streamlit run app.py
```

* Admin panel: Add playlists & courses
* Frontend: Select course → view playlist or video

---

✅ **Features implemented:**

* [x] Admin panel to manage playlists & courses
* [x] SQLite database for persistent storage
* [x] Frontend course selection with playlist sidebar
* [x] Handles single videos or playlists
* [x] API key stored securely in `secrets.toml`

---

I can **upgrade this next** to:

* Show **video description & top comments**
* Display **playlist thumbnails in a card layout**
* Add **batch filtering** in frontend

This will make it a **fully polished mini-LMS** like a real course platform.

Do you want me to add that next?
